In [60]:
import re
import string
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata

from itertools import cycle
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import KeyedVectors
from gensim.models import FastText

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, Dropout, Dense, Masking

import nltk.corpus
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from indoNLP.preprocessing import replace_slang

import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to C:\Users\TUF
[nltk_data]     Gaming\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\TUF
[nltk_data]     Gaming\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\TUF
[nltk_data]     Gaming\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
df = pd.read_csv('myxl_reviews.csv')
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,4c669b18-5f47-4ef6-8824-681f545daf15,Gie Ozora,https://play-lh.googleusercontent.com/a/ACg8oc...,"tidak rekomen, signal hanya 1 bar,giliran lebi...",1,69,8.0.0,2025-03-26 10:38:20,"Hi Kak, maaf ya, agar dapat dibantu cek lebih ...",2025-03-26 12:00:40,8.0.0
1,376d73cb-f14e-4c97-b7dc-549fcc4d2222,Abdul Syafii,https://play-lh.googleusercontent.com/a-/ALV-U...,aduh makin paraha ajah sih ini makin kesini ma...,2,7,8.1.0,2025-04-01 20:12:28,"Hi Kak, Maya mohon maaf ya atas kendalanya, yu...",2025-04-01 22:01:11,8.1.0
2,d733f9a9-5de4-4ce2-81f4-d13d75930812,Rabun 26,https://play-lh.googleusercontent.com/a/ACg8oc...,Aplikasi nya sering bermasalah jaringan selalu...,4,956,8.0.0,2025-03-10 00:54:48,"Hi Kak, terima kasih atas ulasannya. Apabila a...",2025-03-10 02:01:06,8.0.0
3,4e4ebdb1-2a06-474a-8863-512f8c9c4947,Puji Nugroho,https://play-lh.googleusercontent.com/a-/ALV-U...,"aplikasi sering error, harus login dengan nomo...",1,32,8.1.0,2025-03-27 20:30:20,"Hi Kak, mohon maaf ya atas ketidaknyamanan yan...",2025-03-27 22:01:07,8.1.0
4,3e317d3f-be0f-47fd-8aee-600f56487ef2,Elyse,https://play-lh.googleusercontent.com/a-/ALV-U...,Tolong untuk pihak XL jika update aplikasi itu...,1,129,8.0.0,2025-03-15 19:12:28,"Hi Kak, terima kasih atas kepercayaannya untuk...",2024-05-22 10:01:04,8.0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              20000 non-null  object
 1   userName              20000 non-null  object
 2   userImage             20000 non-null  object
 3   content               20000 non-null  object
 4   score                 20000 non-null  int64 
 5   thumbsUpCount         20000 non-null  int64 
 6   reviewCreatedVersion  17905 non-null  object
 7   at                    20000 non-null  object
 8   replyContent          19091 non-null  object
 9   repliedAt             19091 non-null  object
 10  appVersion            17905 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.7+ MB


In [4]:
print(df.isnull().sum())

reviewId                   0
userName                   0
userImage                  0
content                    0
score                      0
thumbsUpCount              0
reviewCreatedVersion    2095
at                         0
replyContent             909
repliedAt                909
appVersion              2095
dtype: int64


In [5]:
print(df.duplicated().sum())

0


In [6]:
df = df.drop(columns=['reviewId','userName','userImage','score','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','appVersion'])
df.head()

,content
0,"tidak rekomen, signal hanya 1 bar,giliran lebi..."
1,aduh makin paraha ajah sih ini makin kesini ma...
2,Aplikasi nya sering bermasalah jaringan selalu...
3,"aplikasi sering error, harus login dengan nomo..."
4,Tolong untuk pihak XL jika update aplikasi itu...


In [7]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text) # hapus simbol
    # remove koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
    text = re.sub(r'([a-zA-Z])\1\1','\\1', text)
    # remove ASCII dan unicode
    text = text.encode('ascii', 'ignore').decode('utf-8')
    text = re.sub(r'[^\x00-\x7f]',r'', text)
    text = re.sub(r'^[ ]|[ ]$','', text) # remove spasi di awal dan akhir kalimat
    text= re.sub(r'\s+', ' ', text).strip() # remove whitespaces
    text = re.sub(' +', ' ', text) # remove spasi ganda (atau lebih) menjadi satu spasi
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text

def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')

def fix_slangwords(text):
    try:
        text = normalize_text(text)
        return replace_slang(text)
    except KeyError:
        return text
        
def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text

def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    # Memecah teks menjadi daftar kata
    words = text.split()
    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]
    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [8]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
df['text_clean'] = df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)

df['text_normalize'] = df['text_casefoldingText'].apply(normalize_text)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
df['text_slangwords'] = df['text_normalize'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
df['text_akhir'] = df['text_stopword'].apply(toSentence)

In [9]:
df.tail()

,content,text_clean,text_casefoldingText,text_normalize,text_slangwords,text_tokenizingText,text_stopword,text_akhir
19995,"Sudah update, app sering tidak merespon, kadan...",Sudah update app sering tidak merespon kadang ...,sudah update app sering tidak merespon kadang ...,sudah update app sering tidak merespon kadang ...,sudah update app sering tidak merespon kadang ...,"[sudah, update, app, sering, tidak, merespon, ...","[update, app, merespon, kadang, lihat, sisa, k...",update app merespon kadang lihat sisa kuota in...
19996,aplikasinya sii bagus dan sangat membantu tapi...,aplikasinya sii bagus dan sangat membantu tapi...,aplikasinya sii bagus dan sangat membantu tapi...,aplikasinya sii bagus dan sangat membantu tapi...,aplikasinya sih bagus dan sangat membantu tapi...,"[aplikasinya, sih, bagus, dan, sangat, membant...","[aplikasinya, bagus, membantu, sinyalnya, lho,...",aplikasinya bagus membantu sinyalnya lho jelek...
19997,to the point aja ngasih alasannya. kenapa gak ...,to the point aja ngasih alasannya kenapa gak m...,to the point aja ngasih alasannya kenapa gak m...,to the point aja ngasih alasannya kenapa gak m...,tapi the point saja mengasih alasannya kenapa ...,"[tapi, the, point, saja, mengasih, alasannya, ...","[point, mengasih, alasannya, login, masukkan, ...",point mengasih alasannya login masukkan kode r...
19998,Saya kasih bintang 1 karena aplikasi XL di say...,Saya kasih bintang karena aplikasi XL di saya ...,saya kasih bintang karena aplikasi xl di saya ...,saya kasih bintang karena aplikasi xl di saya ...,saya kasih bintang karena aplikasi xl di saya ...,"[saya, kasih, bintang, karena, aplikasi, xl, d...","[kasih, bintang, aplikasi, xl, mahal, isi, pul...",kasih bintang aplikasi xl mahal isi pulsa maha...
19999,Jaringan tidak stabil patah patah untuk bermai...,Jaringan tidak stabil patah patah untuk bermai...,jaringan tidak stabil patah patah untuk bermai...,jaringan tidak stabil patah patah untuk bermai...,jaringan tidak stabil patah patah untuk bermai...,"[jaringan, tidak, stabil, patah, patah, untuk,...","[jaringan, stabil, patah, patah, bermain, mobi...",jaringan stabil patah patah bermain mobile leg...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   content               20000 non-null  object
 1   text_clean            20000 non-null  object
 2   text_casefoldingText  20000 non-null  object
 3   text_normalize        20000 non-null  object
 4   text_slangwords       20000 non-null  object
 5   text_tokenizingText   20000 non-null  object
 6   text_stopword         20000 non-null  object
 7   text_akhir            20000 non-null  object
dtypes: object(8)
memory usage: 1.2+ MB


In [11]:
df_original = df.copy()

In [12]:
# Load model dan tokenizer
model_name = "w11wo/indonesian-roberta-base-sentiment-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [13]:
# Mapping label (disesuaikan dari metadata model)
id2label = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

In [14]:
# Fungsi prediksi
def predict_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral"  # atau label default lain, misal "netral"

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    inputs = {k: v.to(torch.long) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

        if outputs.logits.shape[1] == 0:
            return "neutral"

        predicted_class_id = torch.argmax(outputs.logits, dim=1).item()
        return id2label[predicted_class_id]


In [15]:
df_labeled = df_original.copy()

In [16]:
# Terapkan ke kolom DataFrame
df_labeled['label'] = df_labeled['text_akhir'].apply(predict_sentiment)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [18]:
df_labeled.head()

,content,text_clean,text_casefoldingText,text_normalize,text_slangwords,text_tokenizingText,text_stopword,text_akhir,label
0,"tidak rekomen, signal hanya 1 bar,giliran lebi...",tidak rekomen signal hanya bargiliran lebih da...,tidak rekomen signal hanya bargiliran lebih da...,tidak rekomen signal hanya bargiliran lebih da...,tidak rekomen signal hanya bargiliran lebih da...,"[tidak, rekomen, signal, hanya, bargiliran, le...","[rekomen, signal, bargiliran, bar, koneksi, in...",rekomen signal bargiliran bar koneksi internet...,positive
1,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha saja sih ini makin kesini ma...,"[aduh, makin, paraha, saja, sih, ini, makin, k...","[aduh, paraha, kesini, rekomen, signal, bar, g...",aduh paraha kesini rekomen signal bar giliran ...,positive
2,Aplikasi nya sering bermasalah jaringan selalu...,Aplikasi nya sering bermasalah jaringan selalu...,aplikasi nya sering bermasalah jaringan selalu...,aplikasi nya sering bermasalah jaringan selalu...,aplikasi nya sering bermasalah jaringan selalu...,"[aplikasi, nya, sering, bermasalah, jaringan, ...","[aplikasi, bermasalah, jaringan, lag, bermain,...",aplikasi bermasalah jaringan lag bermain game ...,positive
3,"aplikasi sering error, harus login dengan nomo...",aplikasi sering error harus login dengan nomor...,aplikasi sering error harus login dengan nomor...,aplikasi sering error harus login dengan nomor...,aplikasi sering error harus login dengan nomor...,"[aplikasi, sering, error, harus, login, dengan...","[aplikasi, error, login, nomor, server, eror, ...",aplikasi error login nomor server eror lihat n...,positive
4,Tolong untuk pihak XL jika update aplikasi itu...,Tolong untuk pihak XL jika update aplikasi itu...,tolong untuk pihak xl jika update aplikasi itu...,tolong untuk pihak xl jika update aplikasi itu...,tolong untuk pihak xl jika update aplikasi itu...,"[tolong, untuk, pihak, xl, jika, update, aplik...","[tolong, xl, update, aplikasi, perbaiki, kemba...",tolong xl update aplikasi perbaiki kembangin b...,neutral


In [19]:
print(df_labeled["label"].value_counts())

label
positive    11339
neutral      5791
negative     2870
Name: count, dtype: int64


## Word embedding - fasttext

In [20]:
X = df_labeled['text_akhir']
y = df_labeled['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [21]:
# Format data ke bentuk label dan teks
train_data = '\n'.join([f"__label__{label} {text}" for label, text in zip(y_train, X_train)])
test_data = '\n'.join([f"__label__{label} {text}" for label, text in zip(y_test, X_test)])

with open("train.txt", "w", encoding="utf-8") as f:
    f.write(train_data)

with open("test.txt", "w", encoding="utf-8") as f:
    f.write(test_data)


In [22]:
import fasttext

model_ft = fasttext.train_supervised(input="train.txt", epoch=25, lr=1.0, wordNgrams=2, verbose=2, minCount=1)

In [23]:
model_ft.test("test.txt")

(4000, 0.80075, 0.80075)

In [24]:
def predict(text):
    label = model_ft.predict(text)[0][0]
    return label.replace("__label__", "")

df['predicted_label'] = df['text_akhir'].apply(predict)

In [25]:
# Dapatkan dimensi vektor dari kata apa pun yang ada di model
sample_word = list(model_ft.get_words())[0]
vector_dim = len(model_ft.get_word_vector(sample_word))

In [26]:
# Ambil semua kata dari FastText
def vectorize_text(text, model_ft, vector_dim):
    tokens = text.split()
    vectors = []
    for token in tokens:
        try:
            vectors.append(model_ft.get_word_vector(token))
        except:
            vectors.append(np.zeros(vector_dim))
    return vectors


In [27]:
# Ubah teks menjadi vektor
X_vectors = df_labeled['text_akhir'].apply(lambda x: vectorize_text(str(x), model_ft, vector_dim))

In [28]:
# Padding sequences
max_len = max(X_vectors.apply(len))
X_padded = pad_sequences(X_vectors, maxlen=max_len, padding='post', dtype='float32')

In [29]:
# Encode label
le = LabelEncoder()
y_encoded = le.fit_transform(df_labeled['label'])

In [30]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

In [31]:
# One-hot encode label
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

## Model deep learning

In [32]:
# Build LSTM model
model_lstm = Sequential()
model_lstm.add(Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(LSTM(128, return_sequences=True))
model_lstm.add(Dropout(0.4))
model_lstm.add(LSTM(64, return_sequences=False))
model_lstm.add(Dropout(0.3))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(3, activation='softmax'))

C:\Users\TUF Gaming\miniconda3\envs\main-da\lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [33]:
learning_rate = 0.0005
optimizer = Adam(learning_rate=learning_rate)

model_lstm.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [34]:
# Tampilkan arsitektur model
model_lstm.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ masking (Masking)                    │ (None, 92, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 92, 128)             │         117,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 92, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 171,011 (668.01 KB)

 Trainable params: 171,011 (668.01 KB)

 Non-trainable params: 0 (0.00 B)

In [35]:
# Train model
history = model_lstm.fit(
    X_train, 
    y_train_cat, 
    validation_data=(X_test, y_test_cat),
    epochs=10, 
    batch_size=32,
    verbose=1,
)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 38s 65ms/step - accuracy: 0.8152 - loss: 0.5058 - val_accuracy: 0.8675 - val_loss: 0.3253
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 33s 67ms/step - accuracy: 0.8609 - loss: 0.3550 - val_accuracy: 0.8692 - val_loss: 0.3261
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 67ms/step - accuracy: 0.8657 - loss: 0.3397 - val_accuracy: 0.8737 - val_loss: 0.3121
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8666 - loss: 0.3314 - val_accuracy: 0.8745 - val_loss: 0.3152
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8717 - loss: 0.3213 - val_accuracy: 0.8798 - val_loss: 0.3066
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 35s 70ms/step - accuracy: 0.8723 - loss: 0.3121 - val_accuracy: 0.8805 - val_loss: 0.2996
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8738 - loss: 0.3068 - val_accuracy: 0.8875 - val_loss: 0.2970
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8854 - loss: 0.2942 - 

In [36]:
# Evaluate
loss, accuracy = model_lstm.evaluate(X_test, y_test_cat)
print(f"Test Loss: {loss:.4f} | Test Accuracy: {accuracy:.4f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 0.8841 - loss: 0.3022
Test Loss: 0.2896 | Test Accuracy: 0.8875


In [37]:
# Build GRU model
model_gru = Sequential()
model_gru.add(Masking(mask_value=0., input_shape=(X_train.shape[1], X_train.shape[2])))
model_gru.add(GRU(128, return_sequences=True))
model_gru.add(Dropout(0.4))
model_gru.add(GRU(64, return_sequences=False))
model_gru.add(Dropout(0.3))
model_gru.add(Dense(64, activation='relu'))
model_gru.add(Dense(3, activation='softmax'))

In [38]:
learning_rate = 0.0005
optimizer = Adam(learning_rate=learning_rate)

model_gru.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [39]:
# Train model
history = model_gru.fit(
    X_train, 
    y_train_cat, 
    validation_data=(X_test, y_test_cat),
    epochs=10, 
    batch_size=32,
    verbose=1
)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 65ms/step - accuracy: 0.7962 - loss: 0.5154 - val_accuracy: 0.8612 - val_loss: 0.3551
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 69ms/step - accuracy: 0.8588 - loss: 0.3548 - val_accuracy: 0.8730 - val_loss: 0.3188
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8605 - loss: 0.3480 - val_accuracy: 0.8643 - val_loss: 0.3257
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 69ms/step - accuracy: 0.8675 - loss: 0.3310 - val_accuracy: 0.8773 - val_loss: 0.3111
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 36s 72ms/step - accuracy: 0.8702 - loss: 0.3247 - val_accuracy: 0.8758 - val_loss: 0.3214
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 69ms/step - accuracy: 0.8695 - loss: 0.3282 - val_accuracy: 0.8820 - val_loss: 0.2975
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 68ms/step - accuracy: 0.8671 - loss: 0.3206 - val_accuracy: 0.8832 - val_loss: 0.2978
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 34s 67ms/step - accuracy: 0.8778 - loss: 0.3095 - 

In [40]:
# Evaluate
loss, accuracy = model_gru.evaluate(X_test, y_test_cat)
print(f"Test Loss: {loss:.4f} | Test Accuracy: {accuracy:.4f}")

125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8702 - loss: 0.3113
Test Loss: 0.2989 | Test Accuracy: 0.8783


## Lexicon

In [41]:
df_clean = df_original.copy()

In [42]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [43]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score > 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
    else:
      polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [44]:
results = df_clean['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df_clean['polarity_score'] = results[0]
df_clean['polarity'] = results[1]
print(df_clean['polarity'].value_counts())

polarity
negative    13104
positive     5973
neutral       923
Name: count, dtype: int64


In [45]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   content               20000 non-null  object
 1   text_clean            20000 non-null  object
 2   text_casefoldingText  20000 non-null  object
 3   text_normalize        20000 non-null  object
 4   text_slangwords       20000 non-null  object
 5   text_tokenizingText   20000 non-null  object
 6   text_stopword         20000 non-null  object
 7   text_akhir            20000 non-null  object
 8   polarity_score        20000 non-null  int64 
 9   polarity              20000 non-null  object
dtypes: int64(1), object(9)
memory usage: 1.5+ MB


In [46]:
df_clean.head()

,content,text_clean,text_casefoldingText,text_normalize,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,"tidak rekomen, signal hanya 1 bar,giliran lebi...",tidak rekomen signal hanya bargiliran lebih da...,tidak rekomen signal hanya bargiliran lebih da...,tidak rekomen signal hanya bargiliran lebih da...,tidak rekomen signal hanya bargiliran lebih da...,"[tidak, rekomen, signal, hanya, bargiliran, le...","[rekomen, signal, bargiliran, bar, koneksi, in...",rekomen signal bargiliran bar koneksi internet...,-13,negative
1,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha ajah sih ini makin kesini ma...,aduh makin paraha saja sih ini makin kesini ma...,"[aduh, makin, paraha, saja, sih, ini, makin, k...","[aduh, paraha, kesini, rekomen, signal, bar, g...",aduh paraha kesini rekomen signal bar giliran ...,-17,negative
2,Aplikasi nya sering bermasalah jaringan selalu...,Aplikasi nya sering bermasalah jaringan selalu...,aplikasi nya sering bermasalah jaringan selalu...,aplikasi nya sering bermasalah jaringan selalu...,aplikasi nya sering bermasalah jaringan selalu...,"[aplikasi, nya, sering, bermasalah, jaringan, ...","[aplikasi, bermasalah, jaringan, lag, bermain,...",aplikasi bermasalah jaringan lag bermain game ...,-12,negative
3,"aplikasi sering error, harus login dengan nomo...",aplikasi sering error harus login dengan nomor...,aplikasi sering error harus login dengan nomor...,aplikasi sering error harus login dengan nomor...,aplikasi sering error harus login dengan nomor...,"[aplikasi, sering, error, harus, login, dengan...","[aplikasi, error, login, nomor, server, eror, ...",aplikasi error login nomor server eror lihat n...,-19,negative
4,Tolong untuk pihak XL jika update aplikasi itu...,Tolong untuk pihak XL jika update aplikasi itu...,tolong untuk pihak xl jika update aplikasi itu...,tolong untuk pihak xl jika update aplikasi itu...,tolong untuk pihak xl jika update aplikasi itu...,"[tolong, untuk, pihak, xl, jika, update, aplik...","[tolong, xl, update, aplikasi, perbaiki, kemba...",tolong xl update aplikasi perbaiki kembangin b...,-4,negative


## Tfidf

In [47]:
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = df_clean['text_akhir']
y = df_clean['polarity']

In [48]:
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=1000, min_df=20, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

In [49]:
# Konversi sparse matrix menjadi array agar bisa diproses oleh SMOTE
X_tfidf_array = X_tfidf.toarray()

In [50]:
!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
# 📌 Terapkan SMOTE untuk menyeimbangkan data
smote = SMOTE(sampling_strategy="auto", random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf_array, y)

C:\Users\TUF Gaming\miniconda3\envs\main-da\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [51]:
# Buat DataFrame dengan fitur yang benar
df_resampled = pd.DataFrame(X_resampled, columns=tfidf.get_feature_names_out())  # GUNAKAN tfidf
df_resampled["label"] = y_resampled  # Tambahkan label ke DataFrame

In [52]:
print(df_resampled["label"].value_counts())

label
negative    13104
positive    13104
neutral     13104
Name: count, dtype: int64


In [53]:
# Pisahkan fitur dan label dari data hasil resampling
X = df_resampled.drop("label", axis=1)
y = df_resampled["label"]

In [54]:
# Bagi data menjadi 80% training dan 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y,      # Agar distribusi label tetap seimbang
    random_state=42
)

## Model machine learning

In [55]:
from sklearn.ensemble import RandomForestClassifier

# Membuat objek model Random Forest
random_forest = RandomForestClassifier(
    n_estimators=800,         # Kurangi jumlah pohon
    max_depth=None,             # Batasi kedalaman pohon
    min_samples_split=10,      # Minimum sampel untuk split
    min_samples_leaf=1,       # Minimum sampel di setiap leaf
    random_state=0
)

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9966612610893828
Random Forest - accuracy_test: 0.9116113442706346


In [56]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Membuat objek model SVM
svm_model = SVC(C=10, kernel='linear')  # Gunakan kernel linear untuk teks

# Melatih model SVM pada data pelatihan
svm_model.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_svm = svm_model.predict(X_train)
y_pred_test_svm = svm_model.predict(X_test)

# Evaluasi akurasi model SVM
accuracy_train_svm = accuracy_score(y_pred_train_svm, y_train)
accuracy_test_svm = accuracy_score(y_pred_test_svm, y_test)

# Menampilkan akurasi
print('SVM - accuracy_train:', accuracy_train_svm)
print('SVM - accuracy_test:', accuracy_test_svm)

SVM - accuracy_train: 0.9481700531018474
SVM - accuracy_test: 0.9235660689304337


In [57]:
# Bagi data menjadi data latih 30% dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [58]:
random_forest = RandomForestClassifier(
    n_estimators=800,         # Kurangi jumlah pohon
    max_depth=None,             # Batasi kedalaman pohon
    min_samples_split=10,      # Minimum sampel untuk split
    min_samples_leaf=1,       # Minimum sampel di setiap leaf
    random_state=0
)

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train, y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train)
y_pred_test_rf = random_forest.predict(X_test)

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9974925503306926
Random Forest - accuracy_test: 0.9096998473800237


In [59]:
# Input kalimat baru dari pengguna
kalimat_baru = input("Masukkan kalimat baru: ")

# Melakukan preprocessing pada kalimat baru
kalimat_baru_cleaned = cleaningText(kalimat_baru)
kalimat_baru_casefolded = casefoldingText(kalimat_baru_cleaned)
kalimat_baru_slangfixed = fix_slangwords(kalimat_baru_casefolded)
kalimat_baru_tokenized = tokenizingText(kalimat_baru_slangfixed)
kalimat_baru_filtered = filteringText(kalimat_baru_tokenized)
kalimat_baru_final = toSentence(kalimat_baru_filtered)

# Menggunakan objek tfidf yang sudah di-fit dari pelatihan sebelumnya
X_kalimat_baru = tfidf.transform([kalimat_baru_final]).toarray()

# Memperoleh prediksi sentimen kalimat baru
prediksi_sentimen = svm_model.predict(X_kalimat_baru)

# Tampilkan hasil
hasil = prediksi_sentimen[0]
if hasil == 'positive':
    print("Sentimen kalimat baru adalah POSITIF.")
elif hasil == 'neutral':
    print("Sentimen kalimat baru adalah NETRAL.")
else:
    print("Sentimen kalimat baru adalah NEGATIF.")

Masukkan kalimat baru:  aplikasinya jelek


Sentimen kalimat baru adalah NEGATIF.


C:\Users\TUF Gaming\miniconda3\envs\main-da\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
